<a href="https://colab.research.google.com/github/logicmohe/Document-Automation/blob/master/OLG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sample Calculate
Use historical lotto max data as practise for python, also shows how low is the winning rate.

https://en.wikipedia.org/wiki/Lotto_Max

In [ ]:
# Win Rate
# Lotto Max 7 number + bonus
class rate():
  def __init__(self, fund):
    '''
    :param fund: total fund of the session
    '''
    self._fund = fund
  def balance(self, prize, odd):
    total = 0
    for i in range(len(prize)):
      total += (prize[i]*odd[i])
    return total
  def match (self, bonus):
    prize, odd = [], []
    if bonus == True:
      prize=[self._fund*0.8725, self._fund*0.025,
             self._fund*0.015, self._fund*0.0275, 20]
      odd = [1/33294800, 1/4756400, 1/37749,
             1/1104, 1/89.9]
      return self.balance(prize, odd)
    else:
      prize=[self._fund*0.8725, self._fund*0.025,
             self._fund*0.035, 20, 5]
      odd = [1/33294800, 1/113248, 1/1841,
             1/82.9, 1/8.5]
      return self.balance(prize, odd)

In [ ]:
lotto=rate(15000000)
print("Buy Lotto Max without bonus")
print("Absolute Return: " + str(lotto.match(False)) + "\n"
      "Return Rate: " + str(lotto.match(False)/5))
print("Buy Lotto Max with bonus")
print("Absolute Return: " + str(lotto.match(True)) + "\n"
      "Return Rate: " + str(lotto.match(True)/6))

Buy Lotto Max without bonus
Absolute Return: 289.70498863516883
Return Rate: 57.94099772703377
Buy Lotto Max with bonus
Absolute Return: 380.2961170910833
Return Rate: 63.38268618184722


In [ ]:
from fractions import Fraction
a=(7/50 * 6/49 * 5/48 * 4/47 * 3/46 * 2/45 * 1/44)
print(1/a) 

99884399.99999999


In [ ]:
from math import factorial
#https://garsia.math.yorku.ca/~zabrocki/math5020f03/lot649/lot649v3.pdf
print(factorial(5))
def set(n, r):
  return ((factorial(n)/(factorial(r)*factorial(n-r))))

print(set(6,4)*set(43,2)/set(49,6))
# total,totalwin / totalwin, actualwin * total,(totalwin-actualwin)
print(set(50,7)/(set(7,7)*set(50,0)))

120
0.000968619724401408
99884400.0


In [ ]:
# 2021-04-27
# 7/7 0	$10,000,000.00
# 6/7 + Bonus 1	      $133,018.70
# 6/7         34	    $3,912.30
# 5/7 + Bonus 94	    $849.10
# 5/7         1,889	  $98.60
# 4/7 + Bonus 3,454	  $42.40
# 4/7         42,744	$20.00
# 3/7 + Bonus 43,917	$20.00
# 3/7         407,757	FREE PLAY
winningRate = 1/7
people = [1, 34, 94, 1889, 3454, 42744, 43744, 43917, 407757]
prize = [133018, 3912, 849, 98, 42, 20, 20, 5]


In [ ]:
# https://www.python-course.eu/python3_lambda.php
totalTicket = (sum(people)/3)/winningRate
totalMoney = totalTicket*5
totalPrize = sum(map(lambda x,y: x*y, prize,people))
print(sum(map(lambda x: x, prize)))
from functools import reduce
print(reduce(lambda x,y: x+y, prize))
print(totalMoney)
print(totalPrize)
print("Total absolution return")
print(totalPrize/totalMoney)
# totalTicket = totalWinner / winningRate
# print(totalWinner)
# buyMoney = totalTicket * 5 / 3
# winMoney = 1 * 133018 + 34 * 

137964
137964
6342396.666666668
2625367
Total absolution return
0.4139392627077355


# Max Lotto

$5 each draw, one selected number + two random number

*   Start June 2020, draw frequency doubled
*   Start May 2019, 49 number changed to 50 number

In [ ]:
# Historical Max Lotto Number
import re # 正则表达式 Hold for later attempts
import bs4
from lxml import etree
from collections import deque
from urllib.parse import urljoin

import requests

!pip install varname
from varname import nameof

import itertools

In [ ]:
class Number:
  ''' Draw of number '''
  # Would that be better to have a tuple, instead of class?
  date = ''
  number = []
  bonus = 0
  def __init__(self, date, number, bonus):
    self.date = date
    self.number = number
    self.bonus = bonus

In [ ]:
class GetHistoricalData:
  ''' Collect historical data '''
  def __init__(self, year, type):
    '''
    :param year selected year
    :param type type of lotto (max|649)
    '''
    self._year = str(year)
    self._type = type
    self.connection()
    self.extract()
  def connection(self):
    ''' connect to the target url '''

    self.r = requests.get(
    url = f'https://www.getmylottoresults.com/files/'+self._type+self._year+'.htm',
    headers = {
        'Referer': 'https://www.getmylottoresults.com/lotto-'+self._type+'-past-winning-numbers/',
        'User-Agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 13_2_3 like Mac OS X)'
                      'AppleWebKit/605.1.15 (KHTML, like Gecko)'
                      'Version/13.0.3 Mobile/15E148 Safari/604.1'
        }
    )
  def extract(self):
    ''' extract the yearly data from the selected url '''
    soup = bs4.BeautifulSoup(self.r.text, 'lxml')
    elements = soup.find_all('tr')
    self.year_round=[]
    for element in elements:
      records = element.select('th')
      numbers = element.select('td')
      list_number = []
      if len(numbers) :
        for number in numbers:
          list_number.append(int(number.text))
        if self._type == 'max':
          number_object = Number(records[0].text, list_number[:-1], list_number.pop())
        elif self._type == '649': # Add bonus to make 7 number for testing purpose
          number_object = Number(records[0].text, list_number[:-1], list_number.pop())
        self.year_round.append(number_object)

In [ ]:
total_649_data = [] # Data from each year
for year in range(1982, 2021+1):
  total_649_data.append(GetHistoricalData(year, '649'))

In [ ]:
total_max_data = [] # Data from each year
for year in range(2009, 2021+1):
  total_max_data.append(GetHistoricalData(year, 'max'))

In [ ]:
def check_duplicate(numbers):
  total_set = []
  num_counter = 0
  dup_counter = 0
  for number in numbers:
    num_counter += 1
    if number not in total_set:
      total_set.append(number)
    else:
      dup_counter = dup_counter + 1
      print(num_counter, end=' - ')
      print(number, end = ' ')
      pass
  if (len(total_set) == len(numbers)):
    print('No duplicated number in the history\n')
  else:
    print('\nThere are ' + str(dup_counter) + ' set of numbers duplicated\n')

def get_total_combo(history_data):
  ''' Total combination of numbers, include the bonus number '''
  total_combo = []
  for year in history_data:
    for obj in year.year_round:
      with_bonus = obj.number.copy()
      with_bonus.append(obj.bonus)
      total_combo.append(with_bonus)
  print(len(total_combo))
  check_duplicate(total_combo)
  num_count = [0] * 50
  for combo in total_combo:    
    for num in combo:
      num_count[num - 1] += 1
  for i in range(1,51):
    pass
    #print('Index ' + str(i)+ " " + str(num_count[i-1]))
  return total_combo

def get_reduced_combo(total_data, num_length):
  ''' 
  Get reduced list
  :param total_data: unprocessed data list of number list
  :param num_length: length of number list
  '''
  total_combo = []
  for data in total_data:
    for num in itertools.combinations(data, num_length):
      total_combo.append(list(num))
  print(len(total_combo))
  check_duplicate(total_combo)
  return total_combo



In [ ]:
print(("total_max_eight_combi"))
total_max_eight_combi = get_total_combo(total_max_data)

total_max_eight_combi
661
No duplicated number in the history



In [ ]:

print(("total_max_seven_combo"))
total_max_seven_combo = get_reduced_combo(total_max_eight_combi, 7)

print(("total_max_six_combo"))
total_max_six_combo = get_reduced_combo(total_max_eight_combi, 6)

print(("total_max_five_combo"))
total_max_five_combo = get_reduced_combo(total_max_eight_combi, 5)

print("total_max_four_combo")
total_max_four_combo = get_reduced_combo(total_max_eight_combi, 4)




total_max_seven_combo
5288
No duplicated number in the history

total_max_six_combo
18508
16782 - [8, 9, 19, 31, 46, 48] 
There are 1 set of numbers duplicated

total_max_five_combo
37016
742 - [5, 15, 25, 38, 46] 5956 - [5, 6, 26, 34, 49] 9817 - [3, 6, 34, 42, 45] 10542 - [6, 19, 25, 35, 36] 11270 - [5, 7, 15, 45, 47] 11710 - [2, 14, 15, 24, 40] 13018 - [8, 30, 32, 42, 17] 13246 - [2, 5, 38, 46, 33] 13420 - [8, 11, 20, 28, 36] 13657 - [9, 19, 24, 30, 31] 13743 - [7, 24, 30, 35, 36] 14170 - [2, 5, 10, 11, 25] 14301 - [12, 16, 30, 32, 35] 15417 - [5, 8, 22, 39, 45] 15605 - [15, 24, 25, 27, 43] 15704 - [12, 28, 34, 45, 48] 16103 - [10, 26, 34, 43, 47] 17006 - [7, 11, 19, 34, 45] 17399 - [37, 38, 40, 43, 46] 17440 - [7, 14, 15, 30, 41] 17798 - [8, 10, 24, 26, 41] 17949 - [6, 18, 25, 44, 46] 18147 - [6, 29, 30, 31, 45] 18302 - [9, 19, 28, 31, 46] 18445 - [2, 27, 32, 38, 39] 18787 - [4, 29, 39, 44, 46] 19816 - [12, 28, 36, 45, 26] 20199 - [6, 8, 17, 28, 34] 20252 - [8, 10, 24, 26, 37] 20856

In [ ]:
print(("total_649_seven_combo"))
total_649_seven_combo = get_total_combo(total_649_data)

print(("total_649_six_combo"))
total_649_six_combo = get_reduced_combo(total_649_seven_combo, 6)

print(("total_649_five_combo"))
total_649_five_combo = get_reduced_combo(total_649_seven_combo, 5)

print("total_649_four_combo")
total_649_four_combo = get_reduced_combo(total_649_seven_combo, 4)
# print("total_max_three_combo")
# total_max_three_combo = get_reduced_combo(total_max_eight_combi, 3)

# total_649_combo = get_total_number(total_649_data)

total_649_seven_combo
3894
No duplicated number in the history

total_649_six_combo
27258
14398 - [4, 7, 18, 46, 47, 36] 16022 - [1, 30, 36, 42, 43, 23] 17183 - [8, 16, 32, 40, 46, 28] 17622 - [8, 9, 12, 20, 45, 39] 19760 - [15, 28, 32, 35, 42, 45] 23147 - [6, 14, 29, 38, 46, 5] 25215 - [19, 21, 28, 30, 39, 46] 25370 - [3, 14, 36, 38, 40, 23] 
There are 8 set of numbers duplicated

total_649_five_combo
81774
2615 - [2, 41, 42, 44, 47] 4610 - [19, 24, 36, 38, 41] 4720 - [21, 24, 25, 39, 49] 5057 - [7, 9, 21, 38, 45] 8921 - [18, 19, 31, 45, 41] 9112 - [13, 27, 35, 38, 44] 9344 - [5, 23, 35, 45, 48] 9773 - [8, 22, 31, 32, 39] 12129 - [1, 34, 42, 44, 49] 13990 - [9, 27, 28, 37, 46] 14772 - [2, 11, 34, 44, 49] 15509 - [7, 17, 19, 23, 39] 15747 - [36, 40, 43, 49, 33] 16367 - [4, 8, 39, 40, 46] 16450 - [11, 18, 33, 47, 49] 16550 - [16, 18, 20, 26, 38] 16982 - [8, 22, 43, 45, 16] 17138 - [9, 11, 21, 28, 37] 18198 - [9, 21, 28, 30, 1] 19470 - [8, 17, 25, 26, 42] 19755 - [25, 32, 33, 42, 31] 200

In [ ]:
total_seven_combo = total_649_seven_combo + total_max_seven_combo
check_duplicate(total_seven_combo)
total_six_combo = total_649_six_combo + total_max_six_combo
check_duplicate(total_six_combo)
total_five_combo = total_649_five_combo + total_max_five_combo
# check_duplicate(total_five_combo)
total_four_combo = total_649_four_combo + total_max_four_combo
# check_duplicate(total_four_combo)

No duplicated number in the history

14398 - [4, 7, 18, 46, 47, 36] 16022 - [1, 30, 36, 42, 43, 23] 17183 - [8, 16, 32, 40, 46, 28] 17622 - [8, 9, 12, 20, 45, 39] 19760 - [15, 28, 32, 35, 42, 45] 23147 - [6, 14, 29, 38, 46, 5] 25215 - [19, 21, 28, 30, 39, 46] 25370 - [3, 14, 36, 38, 40, 23] 35363 - [13, 16, 21, 35, 46, 18] 36410 - [9, 14, 19, 28, 46, 48] 37255 - [12, 21, 28, 37, 40, 43] 38106 - [5, 12, 26, 29, 31, 40] 38974 - [8, 12, 34, 37, 44, 45] 44040 - [8, 9, 19, 31, 46, 48] 
There are 14 set of numbers duplicated



In [ ]:
print((50*49*48/6))
a = []
for i in range(1,51):
  a.append(i)
print(a)
b = []
for num in itertools.combinations(a, 4):
  b.append(num)
print(len(b))

19600.0
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]
230300


In [ ]:
def get_combo_for_three():
  ''' 
  Not included for possiblities
  Total 19600 combination of three numbers
  14405 combinations appears twice or more
  2955  combinations appears once
  5195  combinations never appears
  '''
  print((50*49*48/6))
  a = []
  for i in range(1,51):
    a.append(i)
  print(a)
  b = []
  for num in itertools.combinations(a, 3):
    b.append(num)
  print(len(b)) #19600
  print(36960 - 14405*2)



total_max_eight_combi
660
No duplicated number in the history

total_max_seven_combo
5280
No duplicated number in the history

total_max_six_combo
18480
16782 - [8, 9, 19, 31, 46, 48] 
There are 1 set of numbers duplicated

total_max_five_combo
36960
There are 2094 set of numbers duplicated

total_max_four_combo
46200
There are 2094 set of numbers duplicated

total_max_three_combo
36960
There are 14405 set of numbers duplicated



In [ ]:
def check_exist(targets, total_combo):
  count = 0
  for target in targets:
    print(target)
    if target in total_combo:
      count += 1
      print("Exist already")
  else:
    print('Not Exist')
  print(count)

def large_small_side(total_combo):
  large_counter, small_counter = 0, 0
  for number in total_combo:
    large_cnt, small_cnt = 0,0
    for num in number:
      if num > 25:
        large_cnt = large_cnt + 1
      else:
        small_cnt = small_cnt + 1
    if large_cnt ==7:
      print(number, end='')
      large_counter += 1
    elif small_cnt ==7:
      print(number, end='')
      small_counter +=1
  print('\nOn large side number is ' + str(large_counter) + '; On small side number is '+str(small_counter))


selected = [
            [2, 14, 19, 21, 22, 44, 50],
            ]

check_exist(selected, total_seven_combo)
check_exist(get_reduced_combo(selected, 6), total_six_combo)
check_exist(get_reduced_combo(selected, 5), total_five_combo)
for select in selected:
  check_exist(get_reduced_combo(select, 4), total_four_combo)


[11, 15, 19, 22, 28, 33, 34]
[4, 18, 21, 26, 27, 35, 46]
[5, 10, 12, 21, 25, 28, 31]
[2, 14, 19, 21, 22, 44, 50]
[2, 4, 25, 30, 32, 39, 49]
Not Exist
0
35
No duplicated number in the history

[11, 15, 19, 22, 28, 33]
[11, 15, 19, 22, 28, 34]
[11, 15, 19, 22, 33, 34]
[11, 15, 19, 28, 33, 34]
[11, 15, 22, 28, 33, 34]
[11, 19, 22, 28, 33, 34]
[15, 19, 22, 28, 33, 34]
[4, 18, 21, 26, 27, 35]
[4, 18, 21, 26, 27, 46]
[4, 18, 21, 26, 35, 46]
[4, 18, 21, 27, 35, 46]
[4, 18, 26, 27, 35, 46]
[4, 21, 26, 27, 35, 46]
[18, 21, 26, 27, 35, 46]
[5, 10, 12, 21, 25, 28]
[5, 10, 12, 21, 25, 31]
[5, 10, 12, 21, 28, 31]
[5, 10, 12, 25, 28, 31]
[5, 10, 21, 25, 28, 31]
[5, 12, 21, 25, 28, 31]
[10, 12, 21, 25, 28, 31]
[2, 14, 19, 21, 22, 44]
[2, 14, 19, 21, 22, 50]
[2, 14, 19, 21, 44, 50]
[2, 14, 19, 22, 44, 50]
[2, 14, 21, 22, 44, 50]
[2, 19, 21, 22, 44, 50]
[14, 19, 21, 22, 44, 50]
[2, 4, 25, 30, 32, 39]
[2, 4, 25, 30, 32, 49]
[2, 4, 25, 30, 39, 49]
[2, 4, 25, 32, 39, 49]
[2, 4, 30, 32, 39, 49]
[2, 25, 30,